In [20]:
from extras.fileOperations import getFileNamesInDir
from compmusic.extractors.makam import scoremetadata as md
import os
import sys
import codecs
import json
import pdb

In [24]:
txtfolder = '/Volumes/SHARED/data/SymbTr/txt'
scorename = 'huzzam--kanto--aksak--bana_bir--lavtaci_hristo'
scorefile = os.path.join(txtfolder, scorename+'.txt')

metadata = md.extract(scorefile, scorename)
metadata

{'metadata': {'composer': 'lavtaci_hristo',
  'form': 'kanto',
  'makam': 'huzzam',
  'name': 'bana_bir',
  'sections': [{'endNote': 31, 'name': u'ARANAGME', 'startNote': 1},
   {'endNote': 67, 'name': u'ARANAGME', 'startNote': 32},
   {'endNote': 78, 'name': 'LYRICS_SECTION', 'startNote': 68},
   {'endNote': 98, 'name': 'LYRICS_SECTION', 'startNote': 79},
   {'endNote': 109, 'name': 'LYRICS_SECTION', 'startNote': 99},
   {'endNote': 129, 'name': 'LYRICS_SECTION', 'startNote': 110},
   {'endNote': 143, 'name': 'LYRICS_SECTION', 'startNote': 130},
   {'endNote': 158, 'name': 'LYRICS_SECTION', 'startNote': 144},
   {'endNote': 172, 'name': 'LYRICS_SECTION', 'startNote': 159},
   {'endNote': 191, 'name': 'LYRICS_SECTION', 'startNote': 173},
   {'endNote': 222, 'name': 'LYRICS_SECTION', 'startNote': 192},
   {'endNote': 253, 'name': 'LYRICS_SECTION', 'startNote': 223},
   {'endNote': 267, 'name': 'LYRICS_SECTION', 'startNote': 254},
   {'endNote': 282, 'name': 'LYRICS_SECTION', 'startNote'

In [5]:
txtfolder = '/Volumes/SHARED/data/SymbTr'
[txtfiles, baseTemp, symbtrnames] = getFileNamesInDir(txtfolder, '.txt')
symbtrnames = [os.path.splitext(s)[0] for s in symbtrnames]
logfile = 'logs/log.txt'

metadataFolder = './metadata/'
metadataFiles = [os.path.join(metadataFolder, s + '.json') for s in symbtrnames]

In [6]:
## get section metadata 

# stdout is directed to file to be manipulated later
old_stdout = sys.stdout
sys.stdout = codecs.open(logfile, 'w', "utf-8")

# loop through all scores
for f, mf, s in zip(txtfiles, metadataFiles, symbtrnames):
    if not s[0] == '.': # ignore temprary files
        print s
        metadata = md.extract(f, s)
        with open(mf, 'w') as text_file:
            json.dump(metadata, text_file)
        
# return to old stdout
sys.stdout = old_stdout

print "Extraction complete!"

Extraction complete!


In [7]:
prevLineWarn = False
prevNoLyricsWarn = False
warn_log = []
no_section_log = []
for line in reversed(open(logfile).readlines()):
    if 'Missing section info in lyrics' in line: # no lyrics, no need to copy
        prevNoLyricsWarn = True
        prevLineWarn = False
    elif prevNoLyricsWarn: # score with no lyrics
        if line[0] == ' ': # warning, copy
            prevNoLyricsWarn = True
            prevLineWarn = True
            no_section_log.append(line)
        else:
            prevNoLyricsWarn = False
            prevLineWarn = False
            no_section_log.append(line)
    elif line[0] == ' ': # warning, copy
        prevNoLyricsWarn = False
        prevLineWarn = True
        warn_log.append(line)
    elif prevLineWarn: # score with warning
        prevNoLyricsWarn = False
        prevLineWarn = False
        warn_log.append(line)
    else: # score wo warning
        prevNoLyricsWarn = False
        prevLineWarn = False

with open('logs/warn_log.txt', 'w') as ll:
    ll.writelines(reversed(warn_log))
    
with open('logs/no_section_log.txt', 'w') as ll:
    ll.writelines(reversed(no_section_log))
        

In [8]:
no_section_compact = [n for n in no_section_log for c in ['seyir','aranagme','ornek-oz'] if c not in n]
no_section_compact = []
for n in no_section_log:
    if not ('seyir' in n or 'aranagme' in n or 'ornek-oz' in n or 'taksim' in n or 'etud' in n):
        no_section_compact.append(n)
    
with open('logs/no_section_compact_log.txt', 'w') as ll:
    ll.writelines(reversed(no_section_compact))

In [9]:
import Levenshtein

Levenshtein.distance('huseyni--sarki--agiraksak--bendeni_gordukce--ismet_aga', 'huseyni--sarkiasdfagiraksakadsf--bendeni_gordukce--ismet_aga')

8